# Import

In [50]:
import os
import json
import posixpath
import requests
from PIL import Image

import pandas as pd
import numpy as np

import boto3
import sagemaker

# Set Up

## Global Parameters

In [22]:
# Define Region
region = "us-east-1"

# Define Bucket
bucket = "dsba-6190-final-team-project"
key_level_1 = "imgs"
key_level_2 = "test"

# Define Endpoint
endpoint_name = 'image-classification-drivers-endpoint'

## AWS Functions

In [23]:
# Establish Boto3 Session
boto_sess = boto3.session.Session(region_name=region)

# Invoke SageMaker Runtime Client
runtime = boto_sess.client(service_name='runtime.sagemaker')

# Invoke S3 Client
s3_client = boto_sess.client(service_name='s3')

# Invoke S3 Resource
s3_res = boto_sess.resource('s3')

# Import Test Images
The test images are located in the project S3 Bucket, under with the key prefix **imgs/test**.

## Generate List of Images
First, we create a list of all objects in the project bucket, under prefix **imgs/test**. 

In [24]:
s3_list_obj_param = {
    "Bucket" : bucket,
    "Prefix" : posixpath.join(key_level_1, key_level_2)
}

response = s3_client.list_objects_v2(**s3_list_obj_param)

The initial call returns a large amount of metadata we are not interested in. So we strip it down to just the contents of the the list. The contents still encapsulate a dictionary of file name and additional metadata. So we further strip this down to only extract the file names, aka, the value associated with the key.

We also add a filter to ensure all the contents we are collecting are JPG images.

In [25]:
# Extract Contents from Item Search Response
test_imgs_contents = response['Contents']

# Initialize List of Test Images
test_imgs_list = []

# Append Keys for Test Images
for item in test_imgs_contents:
    for key, value in item.items():
        if key == 'Key' and 'jpg' in value:
            test_imgs_list.append(value)
            
#print(test_imgs_list)

In [26]:
file_name = "test_data.jpg"
s3_res.Bucket(bucket).download_file(test_imgs_list[5], file_name)

In [52]:
#Image(file_name)

In [28]:
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)

In [29]:
response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=payload)
result = response['Body'].read()
result = json.loads(result)
result

[0.0013738921843469143,
 0.006509726867079735,
 0.0007989784935489297,
 0.7969539165496826,
 0.011298753321170807,
 0.0015191242564469576,
 0.1321578323841095,
 0.023936884477734566,
 0.024842921644449234,
 0.0006078389124013484]

# Test REST API

In [71]:
host_url = "https://96gfw8ry96.execute-api.us-east-1.amazonaws.com"
page = "Test/predict"
image_location = posixpath.join("..","imgs", "kaggle","train", "c8", "img_531.jpg")
image_location_test = "test_image\\img_531.jpg"
headers = {
    "Content-Type" : "image/jpg"
}


In [45]:
with open(image_location, 'rb') as f:
    payload = f

FileNotFoundError: [Errno 2] No such file or directory: '/test_image/img_531.jpg'

In [63]:
img = Image.open(image_location)

In [66]:
data = open(image_location,'rb').read()

In [81]:
posixpath.join(host_url, page)

'https://96gfw8ry96.execute-api.us-east-1.amazonaws.com/Test/predict'

In [84]:
response = requests.post(url=posixpath.join(host_url, page),
                  data=data, headers=headers)
json.loads(response.text)

{'payload': [0.0014833195600658655,
  0.004504410549998283,
  0.002835447434335947,
  0.002531913109123707,
  0.0010949309216812253,
  4.184579211141681e-06,
  0.0185650996863842,
  0.009754505008459091,
  0.9559757709503174,
  0.003250404028221965]}